In [102]:
from typing import Sequence, Union

from clingo.symbol import Number, String
from clingo.control import Control
import clingo

In [103]:
def solve(programs: Union[str, Sequence[str]], grounding_context=None, sep:str=' '):
    ctl = clingo.Control(("--models", "0"))
    if isinstance(programs, str):
        ctl.add("base", (), programs)
    else:
        for program in programs:
            ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

## !! above doesn't work

## down under does !!

In [104]:
solve("""

time(0..3).
id(1..2).
state(init;unmoeg_S;initG;unmoegG_S).


obs(comp(ID,State),T+1) :- 
    obs(comp(ID,State),T), 
    time(T),
    not -obs(comp(ID,State),T+1).

-obs(comp(ID,Else),T) :- 
    obs(comp(ID,State),T), 
    State != Else,
    state(Else).


obs(comp(1,init),0).
occ(act(1,destr_S),2).
obs(comp(2,init),0).
occ(act(2,destr_S),0).

obs(comp(ID,New),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,Old),T),
    deo_type(Act,Deo_Type),
    trans(Old,Deo_Type,New).

deo_type(destr_S,unmoeg_S).
trans(init,unmoeg_S,unmoeg_S).

#show obs/2.
%#show occ/2.

""");

Answer 1: { obs(comp(1,init),0) obs(comp(1,init),1) obs(comp(1,init),2) obs(comp(1,unmoeg_S),3) obs(comp(1,unmoeg_S),4) obs(comp(2,init),0) obs(comp(2,unmoeg_S),1) obs(comp(2,unmoeg_S),2) obs(comp(2,unmoeg_S),3) obs(comp(2,unmoeg_S),4) }
SAT 1


In [105]:
solve("""


time(0..3).
id(1..2).
state(init;unmoeg_S;gewaehr;initG;unmoegG_S;gewaehrG).
dual(init,initG).
dual(unmoeg_S,unmoegG_S).
dual(gewaehr,gewaehrG).

gegen(1,2).
gegen(2,1).

obs(comp(ID,State),T+1) :- 
    obs(comp(ID,State),T), 
    time(T),
    not -obs(comp(ID,State),T+1).

-obs(comp(ID,Else),T) :- 
    obs(comp(ID,State),T), 
    State != Else,
    state(Else).

obs(comp(1,init),0).
obs(comp(2,init),0).
%occ(act(2,destr_S),0).
occ(act(2,destr_S),1).
%occ(act(2,destr_S),2).
occ(act(1,geben),2).
occ(act(1,akzep),2).

occ(act(ID, ueberg),T) :-
    occ(act(ID,geben),T),
    occ(act(ID,akzep),T).

obs(comp(ID,New),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,Old),T),
    deo_type(Act,Deo_Type),
    trans(Old,Deo_Type,New),
    not hasDualAndGegenLOcc(ID,New,T).

hasDualAndGegenLOcc(ID,NewState,T) :-
    obs(comp(ID,NewState),T),
    gegen(ID,Gegen),
    occ(act(Gegen,ueberg),T),
    dual(NewState,_).

obs(comp(ID,Dual),T+1) :-
    not occ(act(ID,_),T),
    obs(comp(ID,NowState),T),
    dual(NowState,Dual),
    hasDualAndGegenLOcc(ID,NowState,T).


obs(comp(ID,DualState),T+1) :-
    occ(act(ID,Act),T),
    occ(act(Gegen,ueberg),T),
    gegen(ID,Gegen),
    obs(comp(ID,NowState),T),
    deo_type(Act,Deo_Type),
    trans(NowState,Deo_Type,NewState),
    dual(NewState,DualState).
    %hasDualAndGegenLOcc(ID,NewState,T),
    %gegen(ID,Gegen),
    %occ(act(Gegen,ueberg),T).

deo_type(destr_S,unmoeg_S).
deo_type(ueberg,ueberg).
trans(init,unmoeg_S,unmoeg_S).
trans(init,ueberg,gewaehr).

#show obs/2.
#show occ/2.

%hap(X,T) :- obs(X,T), X = comp(2,State).
%#show hap/2.

#show hasDualAndGegenLOcc/3.

""");

Answer 1: { obs(comp(1,gewaehr),3) obs(comp(1,gewaehr),4) obs(comp(1,init),0) obs(comp(1,init),1) obs(comp(1,init),2) obs(comp(2,init),0) obs(comp(2,init),1) obs(comp(2,unmoegG_S),3) obs(comp(2,unmoegG_S),4) obs(comp(2,unmoeg_S),2) occ(act(1,akzep),2) occ(act(1,geben),2) occ(act(1,ueberg),2) occ(act(2,destr_S),1) hasDualAndGegenLOcc(2,unmoeg_S,2) }
SAT 1


## down under:
#### 2 components, different moments of untergang, but lieferung in der anderen component (vor/gleichzeitig)

In [106]:
solve("""



time(0..3).
id(1..2).
state(init;unmoeg_S;gewaehr;initG;unmoegG_S;gewaehrG).
dual(init,initG).
dual(unmoeg_S,unmoegG_S).
dual(gewaehr,gewaehrG).

gegen(1,2).
gegen(2,1).

obs(comp(ID,State),T+1) :- 
    obs(comp(ID,State),T), 
    time(T),
    not -obs(comp(ID,State),T+1).

-obs(comp(ID,Else),T) :- 
    obs(comp(ID,State),T), 
    State != Else,
    state(Else).

obs(comp(1,init),0).
obs(comp(2,init),0).
%occ(act(2,destr_S),1).
occ(act(2,destr_S),2).
%occ(act(2,destr_S),3).
occ(act(1,geben),2).
occ(act(1,akzep),2).

occ(act(ID, ueberg),T) :-
    occ(act(ID,geben),T),
    occ(act(ID,akzep),T).

obs(comp(ID,New),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,Old),T),
    deo_type(Act,Deo_Type),
    trans(Old,Deo_Type,New),
    not dualAndGegenL(New,ID,T).
    
dualAndGegenL(New,ID,T) :-
    dual(New,_),
    gegen(ID,OtherComp),
    occ(act(OtherComp, ueberg),T).

obs(comp(ID,Dual),T+1) :-
    not occ(act(ID,_),T),
    obs(comp(ID,NowState),T),
    dual(NowState,Dual),
    dualAndGegenL(NowState,ID,T).

obs(comp(ID,DualState),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,NowState),T),
    deo_type(Act,Deo_Type),
    trans(NowState,Deo_Type,NewState),
    dualAndGegenL(NewState,ID,T),
    %gegen(ID,Gegen),
    %occ(act(Gegen,ueberg),T),
    dual(NewState,DualState).

deo_type(destr_S,unmoeg_S).
deo_type(ueberg,ueberg).
trans(init,unmoeg_S,unmoeg_S).
trans(init,ueberg,gewaehr).

trans(DualOri,Type,DualDest) :- 
    dual(Ori,DualOri),
    dual(Dest,DualDest),
    trans(Ori,Type,Dest).

#show obs/2.
#show occ/2.
%#show dualAndGegenL/3.
%#show dual/2.
%#show trans/3.
%#show gegen/2.
%#show occ/2.




%hap(X,T) :- obs(X,T), X = comp(2,State).
%#show hap/2.


""");

Answer 1: { obs(comp(1,gewaehr),3) obs(comp(1,gewaehr),4) obs(comp(1,init),0) obs(comp(1,init),1) obs(comp(1,init),2) obs(comp(2,init),0) obs(comp(2,init),1) obs(comp(2,init),2) obs(comp(2,unmoegG_S),3) obs(comp(2,unmoegG_S),4) occ(act(1,akzep),2) occ(act(1,geben),2) occ(act(1,ueberg),2) occ(act(2,destr_S),2) }
SAT 1


In [107]:
solve("""


time(0..3).

%%%%% Graph

state(init;unmoeg_S;gewaehr;verzug_S;initG;unmoegG_S;gewaehrG;verzugG_S).
dual(init,initG).
dual(unmoeg_S,unmoegG_S).
dual(gewaehr,gewaehrG).
dual(verzug_S,verzugG_S).

trans(init,unmoeg_S,unmoeg_S).
trans(init,ueberg,gewaehr).
trans(init,verzug_S,verzug_S).

trans(DualOri,Type,DualDest) :- 
    dual(Ori,DualOri),
    dual(Dest,DualDest),
    trans(Ori,Type,Dest).
    
    
%%%%%   Action - MetaData

deo_type(destr_S,unmoeg_S).
deo_type(ueberg,ueberg).


%%%%%   Components

id(1..2).

gegen(1,2).
gegen(2,1).






%%%%%   Inertia


obs(comp(ID,State),T+1) :- 
    obs(comp(ID,State),T), 
    time(T),
    not -obs(comp(ID,State),T+1).

-obs(comp(ID,Else),T) :- 
    obs(comp(ID,State),T), 
    State != Else,
    state(Else).


%%%%%     Übergabe

occ(act(ID, ueberg),T) :-
    occ(act(ID,geben),T),
    occ(act(ID,akzep),T).
    

%%%%%    State-Update

obs(comp(ID,New),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,Old),T),
    deo_type(Act,Deo_Type),
    trans(Old,Deo_Type,New),
    not dualAndGegenL(New,ID,T).
    
dualAndGegenL(NewState, CompID,T) :-
    dual(NewState, _),
    gegen(CompID,OtherComp),
    occ(act(OtherComp, ueberg),T).

obs(comp(ID,Dual),T+1) :-
    not occ(act(ID,_),T),
    obs(comp(ID,NowState),T),
    dual(NowState,Dual),
    dualAndGegenL(NowState,ID,T).

obs(comp(ID,DualState),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,NowState),T),
    deo_type(Act,Deo_Type),
    trans(NowState,Deo_Type,NewState),
    dualAndGegenL(NewState,ID,T),
    %gegen(ID,Gegen),
    %occ(act(Gegen,ueberg),T),
    dual(NewState,DualState).





%%%%% Happening

obs(comp(1,init),0).
obs(comp(2,init),0).
%occ(act(2,destr_S),1).
occ(act(2,destr_S),2).
%occ(act(2,destr_S),3).
occ(act(1,geben),2).
occ(act(1,akzep),2).



%%%%%    Filter

#show obs/2.
%#show occ/2.
%#show dualAndGegenL/3.
%#show dual/2.
%#show trans/3.
%#show gegen/2.
%#show occ/2.




%hap(X,T) :- obs(X,T), X = comp(2,State).
%#show hap/2.


""");

Answer 1: { obs(comp(1,gewaehr),3) obs(comp(1,gewaehr),4) obs(comp(1,init),0) obs(comp(1,init),1) obs(comp(1,init),2) obs(comp(2,init),0) obs(comp(2,init),1) obs(comp(2,init),2) obs(comp(2,unmoegG_S),3) obs(comp(2,unmoegG_S),4) }
SAT 1


In [108]:
solve("""


time(0..3).

%%%%% Graph

state(init;unmoeg_S;gewaehr;verzug_S;verzug_G;initG;unmoegG_S;gewaehrG;verzugG_S).
dual(init,initG).
dual(unmoeg_S,unmoegG_S).
dual(gewaehr,gewaehrG).
dual(verzug_S,verzugG_S).


trans(init,unmoeg_S,unmoeg_S).
trans(init,ueberg,gewaehr).
trans(init,verzug_S,verzug_S).
trans(verzug_S,ueberg,gewaehr).
trans(init,verzug_G,verzug_G).


trans(DualOri,Type,DualDest) :- 
    dual(Ori,DualOri),
    dual(Dest,DualDest),
    trans(Ori,Type,Dest).


%%%%%   Action - MetaData

deo_type(destr_S,unmoeg_S).
deo_type(ueberg,ueberg).
deo_type(verzug_S,verzug_S).

%%%%%   Components

id(1..2).

gegen(1,2).
gegen(2,1).


verzugsComp(1,3).
verzugsComp(2,4).


%%%%%   Inertia


obs(comp(ID,State),T+1) :- 
    obs(comp(ID,State),T), 
    time(T),
    not -obs(comp(ID,State),T+1).

-obs(comp(ID,Else),T) :- 
    obs(comp(ID,State),T), 
    State != Else,
    state(Else).


%%%%%   Übergabe

occ(act(ID, ueberg),T) :-
    occ(act(ID,geben),T),
    occ(act(ID,akzep),T).
    
    


%%%%%    State-Update

obs(comp(ID,New),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,Old),T),
    deo_type(Act,Deo_Type),
    trans(Old,Deo_Type,New),
    not dualAndGegenL(New,ID,T).
    
dualAndGegenL(NewState, CompID,T) :-
    dual(NewState, _),
    gegen(CompID,OtherComp),
    occ(act(OtherComp, ueberg),T).

obs(comp(ID,Dual),T+1) :-
    not occ(act(ID,_),T),
    obs(comp(ID,NowState),T),
    dual(NowState,Dual),
    dualAndGegenL(NowState,ID,T).

obs(comp(ID,DualState),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,NowState),T),
    deo_type(Act,Deo_Type),
    trans(NowState,Deo_Type,NewState),
    dualAndGegenL(NewState,ID,T),
    %gegen(ID,Gegen),
    %occ(act(Gegen,ueberg),T),
    dual(NewState,DualState).


    


%%%%%    Filter

#show obs/2.
#show occ/2.
#show dualAndGegenL/3.
#show dual/2.
#show trans/3.
%#show gegen/2.
%#show occ/2.


%hap(X,T) :- obs(X,T), X = comp(2,State).
%#show hap/2.


%%%%%    Verzug

occ(act(ID,verzug_S),T) :-
    obs(comp(ID,init),T),
    time(T),
    deadL(ID,Dead),
    T = Dead-1,
    not occ(act(ID,geben),T).

occ(act(ID,verzug_G),T) :-
    obs(comp(ID,init),T),
    time(T),
    deadL(ID,Dead),
    T = Dead-1,
    occ(act(ID,geben),T),
    not occ(act(ID,akzep),T).



%%%%% Happening

obs(comp(1,init),0).
obs(comp(2,init),0).
%occ(act(2,destr_S),1).
%occ(act(2,destr_S),2).
%occ(act(2,destr_S),3).
%deadL(1,1).
deadL(1,2).
%deadL(1,3).
verzugs_deadL(1,4).
occ(act(1,geben),3).
occ(act(1,akzep),3).
occ(act(2,geben),3).
occ(act(2,akzep),3).

#show gegen/2.
""");




Answer 1: { dual(gewaehr,gewaehrG) dual(init,initG) dual(unmoeg_S,unmoegG_S) dual(verzug_S,verzugG_S) gegen(1,2) gegen(2,1) obs(comp(1,gewaehrG),4) obs(comp(1,init),0) obs(comp(1,init),1) obs(comp(1,verzug_S),2) obs(comp(1,verzug_S),3) obs(comp(2,gewaehrG),4) obs(comp(2,init),0) obs(comp(2,init),1) obs(comp(2,init),2) obs(comp(2,init),3) occ(act(1,akzep),3) occ(act(1,geben),3) occ(act(1,ueberg),3) occ(act(1,verzug_S),1) occ(act(2,akzep),3) occ(act(2,geben),3) occ(act(2,ueberg),3) dualAndGegenL(gewaehr,1,3) dualAndGegenL(gewaehr,2,3) dualAndGegenL(init,1,3) dualAndGegenL(init,2,3) dualAndGegenL(unmoeg_S,1,3) dualAndGegenL(unmoeg_S,2,3) dualAndGegenL(verzug_S,1,3) dualAndGegenL(verzug_S,2,3) trans(init,ueberg,gewaehr) trans(init,unmoeg_S,unmoeg_S) trans(init,verzug_G,verzug_G) trans(init,verzug_S,verzug_S) trans(initG,ueberg,gewaehrG) trans(initG,unmoeg_S,unmoegG_S) trans(initG,verzug_S,verzugG_S) trans(verzugG_S,ueberg,gewaehrG) trans(verzug_S,ueberg,gewaehr) }
SAT 1


In [109]:
solve("""


time(0..3).

%%%%% Graph

state(init;unmoeg_S;gewaehr;verzug_S_gewaehr;verzug_gewaehr;verzug_G;initG;unmoegG_S;gewaehrG;verzugG_S;verzugG_S_gewaehr).
dual(init,initG).
dual(unmoeg_S,unmoegG_S).
dual(gewaehr,gewaehrG).
dual(verzug_S,verzugG_S).
%dual(verzug_S_gewaehr,verzugG_S_gewaehr).


trans(init,unmoeg_S,unmoeg_S).
trans(init,ueberg,gewaehr).
trans(init,verzug_S,verzug_S).
trans(verzug_S,ueberg,verzug_S_gewaehr).
%trans(verzug_S,ueberg,gewaehr).
trans(init,verzug_G,verzug_G).



trans(DualOri,Type,DualDest) :- 
    dual(Ori,DualOri),
    dual(Dest,DualDest),
    trans(Ori,Type,Dest).


%%%%%   Action - MetaData

deo_type(destr_S,unmoeg_S).
deo_type(ueberg,ueberg).
deo_type(verzug_S,verzug_S).

%%%%%   Components

id(1..2).

gegen(1,2).
gegen(2,1).


verzugsComp(1,3).
verzugsComp(2,4).


%%%%%   Inertia


obs(comp(ID,State),T+1) :- 
    obs(comp(ID,State),T), 
    time(T),
    not -obs(comp(ID,State),T+1).

-obs(comp(ID,Else),T) :- 
    obs(comp(ID,State),T), 
    State != Else,
    state(Else).


%%%%%   Übergabe

occ(act(ID, ueberg),T) :-
    occ(act(ID,geben),T),
    occ(act(ID,akzep),T).
    
    


%%%%%    State-Update

obs(comp(ID,New),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,Old),T),
    deo_type(Act,Deo_Type),
    trans(Old,Deo_Type,New),
    not dualAndGegenL(New,ID,T).
    
dualAndGegenL(NewState, CompID,T) :-
    dual(NewState, _),
    gegen(CompID,OtherComp),
    occ(act(OtherComp, ueberg),T).

obs(comp(ID,Dual),T+1) :-
    not occ(act(ID,_),T),
    obs(comp(ID,NowState),T),
    dual(NowState,Dual),
    dualAndGegenL(NowState,ID,T).

obs(comp(ID,DualState),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,NowState),T),
    deo_type(Act,Deo_Type),
    trans(NowState,Deo_Type,NewState),
    dualAndGegenL(NewState,ID,T),
    %gegen(ID,Gegen),
    %occ(act(Gegen,ueberg),T),
    dual(NewState,DualState).


    


%%%%%    Filter

#show obs/2.
#show occ/2.
#show dualAndGegenL/3.
#show dual/2.
#show trans/3.
%#show gegen/2.
%#show occ/2.


%hap(X,T) :- obs(X,T), X = comp(2,State).
%#show hap/2.


%%%%%    Verzug

occ(act(ID,verzug_S),T) :-
    obs(comp(ID,init),T),
    time(T),
    deadL(ID,Dead),
    T = Dead-1,
    not occ(act(ID,geben),T).

occ(act(ID,verzug_G),T) :-
    obs(comp(ID,init),T),
    time(T),
    deadL(ID,Dead),
    T = Dead-1,
    occ(act(ID,geben),T),
    not occ(act(ID,akzep),T).



%%%%% Happening

obs(comp(1,init),0).
obs(comp(2,init),0).
%occ(act(2,destr_S),1).
%occ(act(2,destr_S),2).
%occ(act(2,destr_S),3).
%deadL(1,1).
deadL(1,2).
%deadL(1,3).
verzugs_deadL(1,4).
occ(act(1,geben),3).
%occ(act(1,akzep),3).
%occ(act(2,geben),3).
%occ(act(2,akzep),3).

%#show gegen/2.
""");

Answer 1: { dual(gewaehr,gewaehrG) dual(init,initG) dual(unmoeg_S,unmoegG_S) dual(verzug_S,verzugG_S) obs(comp(1,init),0) obs(comp(1,init),1) obs(comp(1,verzug_S),2) obs(comp(1,verzug_S),3) obs(comp(1,verzug_S),4) obs(comp(2,init),0) obs(comp(2,init),1) obs(comp(2,init),2) obs(comp(2,init),3) obs(comp(2,init),4) occ(act(1,geben),3) occ(act(1,verzug_S),1) trans(init,ueberg,gewaehr) trans(init,unmoeg_S,unmoeg_S) trans(init,verzug_G,verzug_G) trans(init,verzug_S,verzug_S) trans(initG,ueberg,gewaehrG) trans(initG,unmoeg_S,unmoegG_S) trans(initG,verzug_S,verzugG_S) trans(verzug_S,ueberg,verzug_S_gewaehr) }
SAT 1


In [114]:
counter_example = """

time(0..3).

%%%%% Graph

state(init;init_G).
state(unmoegS;unmoegS_G).
state(gewaehr;gewaehr_G).
state(verzugS;verzugS_G).
dual(init,initG).
dual(unmoegS,unmoegS_G).
dual(gewaehr,gewaehrG).
dual(verzugS,verzugS_G).


trans(init,unmoegS,unmoegS).
trans(init,ueberg,gewaehr).
trans(init,verzugS,verzugS).
trans(verzugS,ueberg,verzugS_G).

trans(DualOri,Trans,DualDest) :-
    dual(Ori,DualOri),
    dual(Dest,DualDest),
    trans(Ori,Trans,Dest).

%%%%%   Action - MetaData

deo_type(destrS,unmoegS).
deo_type(ueberg,ueberg).
deo_type(verzugS,verzugS).

%%%% IDs & Components

id(1..2).

gegen(1,2).
gegen(2,1).


verzugsComp(1,3).
verzugsComp(2,4).

%%%%%   Inertia


obs(comp(ID,State),T+1) :-
    obs(comp(ID,State),T),
    time(T),
    not imp(comp(ID,State),T+1).

imp(comp(ID,Else),T) :-
    obs(comp(ID,State),T),
    State != Else,
    state(Else).


%%%%%   Übergabe

occ(act(ID, ueberg),T) :-
    occ(act(ID,geben),T),
    occ(act(ID,akzep),T).




%%%%%    State-Update

obs(comp(ID,New),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,Old),T),
    deo_type(Act,Deo_Type),
    trans(Old,Deo_Type,New),
    not dualAndGegenL(New,ID,T).

dualAndGegenL(NewState, CompID,T) :-
    dual(NewState, _),
    gegen(CompID,OtherComp),
    occ(act(OtherComp, ueberg),T).

obs(comp(ID,Dual),T+1) :-
    not occ(act(ID,_),T),
    obs(comp(ID,NowState),T),
    dual(NowState,Dual),
    dualAndGegenL(NowState,ID,T).

obs(comp(ID,DualState),T+1) :-
    occ(act(ID,Act),T),
    obs(comp(ID,NowState),T),
    deo_type(Act,Deo_Type),
    trans(NowState,Deo_Type,NewState),
    dualAndGegenL(NewState,ID,T),
    dual(NewState,DualState).

%%%% Counterexample:

obs(comp(1,init),0).
obs(comp(2,init),0).
occ(act(1, ueberg), 1).
occ(act(2, destrS), 2).

#show obs/2.
#show occ/2.

counter_example(obs(comp(ID, A), T), obs(comp(ID, B), T)) :- obs(comp(ID, A), T), obs(comp(ID, B), T), A > B.

#show counter_example/2.

"""
solve([counter_example], sep='\n');

Answer 1: {
counter_example(obs(comp(2,unmoegS_G),3),obs(comp(2,initG),3))
obs(comp(1,gewaehr),2)
obs(comp(1,gewaehr),3)
obs(comp(1,gewaehr),4)
obs(comp(1,init),0)
obs(comp(1,init),1)
obs(comp(2,init),0)
obs(comp(2,init),1)
obs(comp(2,initG),2)
obs(comp(2,initG),3)
obs(comp(2,initG),4)
obs(comp(2,unmoegS_G),3)
occ(act(1,ueberg),1)
occ(act(2,destrS),2)
}
SAT 1
